In [13]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import pandas as pd
import matplotlib.pyplot as plt
import PCA
import numpy as np


ModuleNotFoundError: No module named 'PCA'

In [5]:
data_path = ""

In [8]:
def file_to_dataframe(filenames,st = ','): 
    """
    Initialize the DataFrame from a filenames 
    Parameters: 
    -----------
    filenames: str, the name of the file
    st: str, helps to delimites the columns of the datas because "Correspondances_promo
    are delimited with ';' while the other are delimited with ','.
    """
    datas = pd.read_csv(filenames,delimiter=st)
    return datas

In [17]:
def save_to_csv_file(df,filename):
    """
    Save a dataframe on a .csv file at filename
    ----------
    df: dataframe, the DataFrame that you want to save
    filename: str, the place where you want to save your data frame
    """
    df.to_csv(filename, index=False)
    return True

# KMeans

In [11]:

df = file_to_dataframe(data_path + "exemple.csv")


In [ ]:

# Normalisation des données
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# Sélection du nombre de clusters (ici, on suppose K=8 par exemple)
kmeans = KMeans(n_clusters=8, random_state=42)
kmeans.fit(df_scaled)

# Ajout des étiquettes de cluster au DataFrame original
df['Cluster'] = kmeans.labels_

# Évaluation du modèle avec le score de silhouette
score = silhouette_score(df_scaled, kmeans.labels_)
print(f'Score de silhouette: {score}')

# Affichage des résultats
print(df)


# Score de silhouette

In [ ]:
np.random.seed(42)
indices = np.random.choice(range(len(df_scaled)), size=int(len(df_scaled) * 0.1), replace=False)
sample = df_scaled.iloc[indices]
silhouette_scores = []

for k in range(2, 11):  # Testez des valeurs de k de 2 à 10
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(sample)
    score = silhouette_score(sample, kmeans.labels_)
    silhouette_scores.append(score)
    print('Score calculé pour k =', k)

plt.figure(figsize=(8, 6))
plt.plot(range(2, 11), silhouette_scores, marker='o')
plt.xlabel('Nombre de clusters')
plt.ylabel('Score de silhouette KMeans')
plt.title('Score de silhouette pour différents nombres de clusters')
plt.show()


# Methode du Coude

In [ ]:
wcss = []
for i in range(1, 11):  # Test de 1 à 10 clusters
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42)
    kmeans.fit(df_scaled)
    wcss.append(kmeans.inertia_)

# Affichage de la méthode du coude
plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), wcss, marker='o', linestyle='--')
plt.title('Méthode du Coude pour le choix optimal de K')
plt.xlabel('Nombre de clusters')
plt.ylabel('WCSS')
plt.xticks(range(1, 11))
plt.grid(True)
plt.show()

# Analyse par composantes principales 

In [ ]:
pca = PCA(n_components=2)  # Réduction à 2 dimensions pour la visualisation
df_pca = pca.fit_transform(df_scaled)

# Visualisation
plt.figure(figsize=(8, 6))
plt.scatter(df_pca[:, 0], df_pca[:, 1], alpha=0.7)
plt.xlabel('Première composante principale')
plt.ylabel('Deuxième composante principale')
plt.title('Visualisation ACP des données')
plt.show()


# Scale data

In [ ]:
def cluster_data_set(filename, columns, change_inf=np.nan, change_nan=15):
    """
    Load a dataset, and return scaled data.

    Args:
    - filename: Name of the file containing the data.
    - columns: List of column names to include in clustering.
    - change_inf: Value to use for replacing infinity.
    - change_nan: Value to use for replacing NaN.

    Returns:
    - data: DataFrame containing clustered data.
    """
    # Load the dataset from file
    df = file_to_dataframe(filename)
    
    # Select the specified columns
    data = df[columns]
    
    # Replace infinite values with specified value
    data.replace([np.inf, -np.inf], change_inf, inplace=True)

    # Normalize the data
    scaler = StandardScaler()
    datas = scaler.fit_transform(data)
    data = pd.DataFrame(datas, columns=data.columns)

    # Select a random 10% sample of the data
    indices = np.random.choice(range(len(data)), size=int(len(data) * 0.1), replace=False)
    data = data.iloc[indices]

    # Replace NaN values with specified value
    data.replace(np.nan, change_nan, inplace=True)

    return data

# Fonction auxiliaire

In [ ]:
def percent_abo_conditions(df: pd.DataFrame,columns ,cond):
    """
    This function count the number of elements of the condition and group by
    the column 
    """
    series = df.groupby(columns)[cond].count() # Group by specified columns and count occurrences of the condition
    df_filtered_groups = series.reset_index(name = 'POURCENTAGE_' + cond) #Series into DataFrames

    df_filtered_groups = df_filtered_groups.sort_values(by = 'POURCENTAGE_' + cond, ascending=False) # Sort the DataFrame by the count column in descending order

    n = df_filtered_groups['POURCENTAGE_' + cond].sum()
    df_filtered_groups['POURCENTAGE_' + cond] = df_filtered_groups['POURCENTAGE_' + cond] / n * 100

    df_filtered_groups = df_filtered_groups[df_filtered_groups['POURCENTAGE_' + cond] >= 0.01]

    return df_filtered_groups

# Renvoie les informations sur les centres des clusters

In [ ]:
def data_frame_cluster(data, columns, centers_inv, clusters, data_id_abo):
    """
    Create a DataFrame containing clusters, cluster centers, and subscriber IDs.

    Args:
    - data: DataFrame containing the data.
    - columns: List of column names.
    - centers_inv: Inverse cluster centers (denormalized).
    - clusters: Cluster number assigned to each sample.
    - data_id_abo: DataFrame containing subscriber IDs.

    Returns:
    - df_clusters: DataFrame containing clusters, centers, and percentage IDs.
    """
    # Add cluster labels and subscriber IDs to the data
    data['KMEANS'] = clusters
    data['ID_ABONNE'] = data_id_abo['ID_ABONNE']

    # Generate DataFrame with cluster information
    df_clusters = percent_abo_conditions(data, 'KMEANS', 'ID_ABONNE')
    df_clusters = df_clusters.sort_values(by='KMEANS')

    # Round and assign cluster centers to the DataFrame
    centers = np.round(centers_inv, decimals=2)
    for j in range(len(columns)):
        df_clusters[columns[j]] = [centers[i][j] for i in range(len(centers))]

    return df_clusters

In [16]:
from sklearn.metrics import silhouette_samples, silhouette_score
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.cm as cm

# Renvoie des fichiers avec les informations sur les clusters + visualisation des scores de silhouette

In [15]:
def visualize_silhouette_datas_all(filename, columns, range_n_clusters, data_path_results, change_inf=np.nan, change_nan=15):
    """
    Visualize silhouette scores for different numbers of clusters.

    Args:
    - filename: Name of the file containing the data.
    - columns: List of column names to include in clustering.
    - range_n_clusters: List of numbers of clusters to test.
    - data_path_results: Path to save the results.
    - change_inf: Value to use for replacing infinity.
    - change_nan: Value to use for replacing NaN.

    Returns:
    - silhouette_scores: List of silhouette scores.
    """
    # Load the dataset from file
    df = file_to_dataframe(filename)
    data = df[columns]
    
    # Replace infinite values with specified value
    data.replace([np.inf, -np.inf], change_inf, inplace=True)

    # Normalize the data
    scaler = StandardScaler()
    datas = scaler.fit_transform(data)

    data = pd.DataFrame(datas, columns=data.columns)

    # Select a random 10% sample of the data
    indices = np.random.choice(range(len(data)), size=int(len(data) * 0.1), replace=False)
    data = data.iloc[indices]
    data_id_abo = df[['ID_ABONNE']].iloc[indices]

    # Replace NaN values with specified value
    data.replace(np.nan, change_nan, inplace=True)

    silhouette_scores = []

    with PdfPages(data_path_results + "silhouettes_scores.pdf") as pdf:
        for n_clusters in range_n_clusters:

            # Initialize the clusterer with n_clusters value and a random generator
            # seed of 10 for reproducibility.
            clusterer = KMeans(n_clusters, random_state=10)
            clusterer.fit(data)

            centers = clusterer.cluster_centers_
            centers_cluster = np.round(scaler.inverse_transform(centers), decimals=2)

            # Create DataFrame with cluster information
            df_cluster = data_frame_cluster(data, columns, scaler.inverse_transform(centers),
                                             clusterer.labels_, data_id_abo)
            
            # Save CSV 
            save_to_csv_file(df_cluster, data_path_results + "cluster" + str(n_clusters) + ".csv", str(n_clusters))

            cluster_labels = clusterer.fit_predict(data)

            silhouette_avg = silhouette_score(data, cluster_labels)

            silhouette_scores.append(silhouette_avg)

            print(
                "For n_clusters =", n_clusters,
                "The average silhouette_score is :", silhouette_avg,
            )

            fig, ax1 = plt.subplots(1, 1)
            fig.set_size_inches(9, 7)

            ax1.set_xlim([-0.1, 1])
            ax1.set_ylim([0, len(data) + (n_clusters + 1) * 10])

            y_lower = 10
            sample_silhouette_values = silhouette_samples(data, cluster_labels)

            for i in range(n_clusters):

                ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
                ith_cluster_silhouette_values.sort()
                size_cluster_i = ith_cluster_silhouette_values.shape[0]
                y_upper = y_lower + size_cluster_i
                color = cm.nipy_spectral(float(i) / n_clusters)
                ax1.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_silhouette_values, facecolor=color,
                                  edgecolor=color, alpha=0.7)
                ax1.text(-0.8, y_lower + 0.5 * size_cluster_i, str(i))
                y_lower = y_upper + 10

            ax1.set_title("The silhouette plot for the various clusters." + str(n_clusters))
            ax1.set_xlabel("The silhouette coefficient values")
            ax1.set_ylabel("Cluster label")

            ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

            ax1.set_yticks([])
            ax1.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])

            pdf.savefig(fig)
            plt.close(fig)

    return silhouette_scores